# 텍스트 분류 4차 시도 (불용어 50개 적용) 

In [ ]:
import pandas as pd
import numpy as np
import re

In [2]:
from konlpy.tag import Okt, Kkma, Mecab, Hannanum
Okt = Okt()
kkma = Kkma()
hannanum = Hannanum()

In [3]:
df = pd.read_csv('final_text_data2.csv', sep='\t', encoding='UTF-8')

In [4]:
df.columns

Index(['std_clm_type_cd', 'ro_y', 'clm_jgmt_dt', 'ceq_caus_cd', 'ceq_psit_cd',
       'ceq_psit_nm', 'trim_plnt_cd', 'prj_vehl_cd', 'mdl_cd', 'part_sys_cd',
       'repn_part_cd', 'clm_prtl_sbc', '대상 선정', '구분', '책임구', '원인', '검토결과'],
      dtype='object')

In [5]:
df.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
0               W  2017     20170811         ZZ7         B11  조립 문제_미장착/오장착   
1               W  2018     20180822         ZZ7         B11  조립 문제_미장착/오장착   
2               W  2018     20181006         ZZ5         B11  조립 문제_미장착/오장착   
3               W  2018     20180928         ZZ7         B11  조립 문제_미장착/오장착   
4               W  2016     20160526         ZZ5         B11  조립 문제_미장착/오장착   

  trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
0         KV11          CK     J5          TT        82305   
1         KV11          YP     A9          TT        84211   
2         KV11          YP     A9          TT        83912   
3         KV11          YP     A9          TT        83120   
4         KV11          YP     A9          TT        83860   

                                        clm_prtl_sbc 대상 선정   구분  책임구   원인  \
0  [C] 1.현상:엔진룸 다량의 연료 누유 발생.2.점검:엔진룸 다량의 연료 누유 점...     X  NaN  NaN  NaN   
1                       [C] 1.현상: 리어 도어 앤티 취핑 필림 미장착     X  NaN  NaN  NaN   
2  [C] 1.현상:Rr RH 커튼 후크  미장착 출고됨.2.조치내용:Rr RH 커튼 ...     X  NaN  NaN  NaN   
3              [C] 현상및조치   차량출고시 웨즈스트립고무 주름발생 들뜸교환조치     X  NaN  NaN  NaN   
4                     [C] 몰딩 내부 고무 미조립 상태에서 몰딩 조립 출고     X  NaN  NaN  NaN   

   검토결과  
0  X---  
1  X---  
2  X---  
3  X---  
4  X---

In [7]:
df.shape

(10629, 17)

In [9]:
df_O = df.loc[df['대상 선정']=='O']
df_X = df.loc[df['대상 선정']=='X']
df2 = pd.concat([df_O,df_X])
df2.head(3)

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
12               W  2017     20171010         ZZ7         B11  조립 문제_미장착/오장착   
14               W  2018     20180725         ZZ7         B11  조립 문제_미장착/오장착   
15               W  2015     20151114         ZZ3         B11  조립 문제_미장착/오장착   

   trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
12         KV11          CK     J5          AC        91500   
14         KV11          YP     A9          EE        28291   
15         KV11          YP     A9          TT        81750   

                                         clm_prtl_sbc 대상 선정       구분   책임구  \
12  [C] 현상-1. 현상  가. 인도후 오디오 작동시 주파수 및 노이즈 발생으로 고객...     O  기능(커넥터)  생산공장   
14  [C] 1. 현상  주행중 엔진경고등 점등2. 점검내용 및 원인  스캔점검결과 P2...     O  기능(커넥터)  생산공장   
15  [C] 차량 테일게이트트림에서 뜨르르르 이상소음발생으로 테일게이트트림 탈착하여 확인...     O   실내부품이음  생산공장   

     원인                검토결과  
12  미결선  O-기능(커넥터)-생산공장-미결선  
14   기타   O-기능(커넥터)-생산공장-기타  
15   이물    O-실내부품이음-생산공장-이물

In [10]:
df2.shape

(8532, 17)

# 분류현황

In [11]:
category_count = df2['대상 선정'].value_counts()
category_count

X    5519
O    3013
Name: 대상 선정, dtype: int64

In [12]:
category_count = df2['책임구'].value_counts()
category_count

구매      2274
생산공장     724
연구소       14
Name: 책임구, dtype: int64

In [13]:
category_count = df2['원인'].value_counts()
category_count

부품       2274
조립불량      230
결선불량      225
체결불량      103
기타        100
미결선        37
이종/간섭      21
설계/구조      14
이물          8
Name: 원인, dtype: int64

# null인 값 확인 및 제거

In [14]:
df2_bak = df2.copy()

In [15]:
# 제거
# df = df[df['대상 선정'].notnull()].copy()
# 다른값으로 채우기
df2 = df2.fillna('N')

In [16]:
df2.shape

(8532, 17)

In [17]:
category_count = df2['대상 선정'].value_counts()
category_count

X    5519
O    3013
Name: 대상 선정, dtype: int64

In [18]:
category_count = df2['책임구'].value_counts()
category_count

N       5520
구매      2274
생산공장     724
연구소       14
Name: 책임구, dtype: int64

In [19]:
category_count = df2['원인'].value_counts()
category_count

N        5520
부품       2274
조립불량      230
결선불량      225
체결불량      103
기타        100
미결선        37
이종/간섭      21
설계/구조      14
이물          8
Name: 원인, dtype: int64

# 전처리

In [20]:
# 예측값과 실제값 비교를 위해 컬럼을 하나 더 생성합니다.
df2['target_predict'] = df2['대상 선정'].astype(str).copy()
# df2 ['train_charge_predict'] = df2 ['책임구'].astype(str).copy()
# df2 ['train_cause_predict'] = df2 ['원인'].astype(str).copy()

In [21]:
def preprocessing(text):
    # 개행문자 제거
    text= str(text)
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    #text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    #text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [22]:
# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['현상', '[C]','현상및점검','현    상','현 상','점검점검','점검내용','조치내용','점검점검내용','현상및조치','C','원인',
             '점검/원인','사진첨부','(사진첨부)','사진참조','점검및원인','동영상참조','동영상','대리점','고객','고객에게','상기',
             '차량','상기차량은','자동차','당차량은','당차량','당','차량은','점검및조치내용','F/LH','F/RH','L/H','R/H','L/H리어',
             'R/H리어','LH/RH','LH','RH','(RH)','(LH)','R/LH','R/RH','리어RH','리어LH','LH쪽','RH쪽','Rr','RR','FRT','Frt',
             'frt','rr','C-10000','C-100','(좌)','(우)','A/S']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words) #stops에 안 걸리는 나머지 것들 이어붙이기

In [23]:
%time df2['clm_prtl_sbc'] = df2['clm_prtl_sbc'].apply(preprocessing)

Wall time: 109 ms


In [24]:
%time df2['clm_prtl_sbc'] = df2['clm_prtl_sbc'].apply(remove_stopwords)

Wall time: 265 ms


In [25]:
df2['clm_prtl_sbc'].head(10)

12          가  인도후 오디오 작동시 주파수 및 노이즈 발생으로 불만 제기   및 ...
14          주행중 엔진경고등 점등   및  스캔점검결과 P       DPF효율저하...
15      테일게이트트림에서 뜨르르르 이상소음발생으로 테일게이트트림 탈착하여 확인시 클립이...
19                     조수석사이드에어백 경고등 점등됨 컨넥터 체결불량으로 재체결
24                        USB 작동 안됨원인 USB 컨넥터 조립 않됨 재조립
25                       어라운드뷰 장착차량인데 운전석 미러에는 카메라 미장착됨
32        리어LH윈도우작동불량  점검시 뒤운전석쪽만 작동불량 퓨즈확인시정상   트림탈...
38      운전석 뒤도어 유리창 작동 안되어  메인파워스위치 배선인가 점검후뒷도어트림스위치...
40       상기차량 주행중 운전석 시트가 들석이는 현상발생입고 점검시 운전석 앞 우측 시...
58           와이퍼 스몰등 후방카메라 오작동 불량   및   리어 루프 배선 R  ...
Name: clm_prtl_sbc, dtype: object

# 학습데이터와 테스트데이터 만들기

In [26]:
df2 = df2.reindex() #index 재설정. reindex:새로운 색인에 맞도록 객체를 새로 생성하는 기능.

In [27]:
df2.shape

(8532, 18)

In [28]:
split_count = int(df2.shape[0] * 0.9)
split_count

7678

In [29]:
df_train = df2[:split_count].copy()
df_train.shape

(7678, 18)

In [30]:
df_test = df2[split_count:].copy()
df_test.shape

(854, 18)

# 단어 벡터화

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

stops = ['[C]']
vectorizer = CountVectorizer(analyzer = 'word', # 캐릭터 단위로 벡터화 할 수도 있습니다.
                             tokenizer = None, # 토크나이저를 따로 지정해 줄 수도 있습니다.
                             preprocessor = None, # 전처리 도구
                             stop_words = stops, # 불용어 nltk등의 도구를 사용할 수도 있습니다.
                             min_df = 1, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다. 
                             ngram_range=(1, 3), # BOW의 단위를 1~3개로 지정합니다.
                             max_features = 20000 # 만들 피처의 수, 단어의 수가 된다.
                            )
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=['[C]'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [32]:
#%%time 
train_feature_vector = vectorizer.fit_transform(df_train['clm_prtl_sbc'])
train_feature_vector.shape

(7678, 20000)

In [33]:
#%%time 
test_feature_vector = vectorizer.fit_transform(df_test['clm_prtl_sbc'])
test_feature_vector.shape

(854, 20000)

In [34]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
vocab[:10]

20000


['abs',
 'abs 경고등',
 'abs 경고등 점등',
 'ass',
 'ass 교환',
 'avm',
 'avm 수동',
 'avm 수동 보정',
 'avm조정함',
 'a핀에']

In [35]:
dist = np.sum(train_feature_vector, axis=0)

pd.DataFrame(dist, columns=vocab)

abs  abs 경고등  abs 경고등 점등  ass  ass 교환  avm  avm 수동  avm 수동 보정  avm조정함  a핀에  \
0   55        4           5    3      10    3       4          6       3    4   

   ...    히터 작동시 소음  히트  히트 프로텍터  히트 프로텍터 조립불량으로  히트 프로텍터조립불량으로  \
0  ...            3  20        4               3              3   

   히트 프로텍터조립불량으로 엔진  힌지  힌지 우측  힌지 우측 커버  힌지부위며  
0                 4   9      5        25      3  

[1 rows x 20000 columns]

# tf-idf 가중치 적용

In [626]:
# tf-idf(Term Frequency-Inverse Document Frequency). 가중치를 구하는 알고리즘.

In [36]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
transformer

TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True)

In [37]:
#%%time 
train_feature_tfidf = transformer.fit_transform(train_feature_vector)
train_feature_tfidf.shape

(7678, 20000)

In [38]:
#%%time 
test_feature_tfidf = transformer.fit_transform(test_feature_vector)
test_feature_tfidf.shape

(854, 20000)

# 머신러닝 : 학습

In [39]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기를 사용
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=2018)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=2018, verbose=0,
            warm_start=False)

In [40]:
# 학습에 사용할 y_label 을 넣어줍니다.
# 어떤 분야의 청원인지 예측할 것이기 때문에 category를 넣어줍니다.
# y_label2 = df_train['caus_pred']
y_label = df_train['target_predict']
# y_label3 = df_train['charge_pred']
%time forest = forest.fit(train_feature_tfidf, y_label)
# %time forest2 = forest.fit(train_feature_tfidf, y_label2)
# %time forest3 = forest.fit(train_feature_tfidf, y_label3)

Wall time: 2.36 s


# 모델평가

In [41]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

scoring = 'accuracy'
%time score = cross_val_score(forest, train_feature_vector, y_label, cv=k_fold, n_jobs=-1, scoring=scoring)
score

Wall time: 33.7 s


array([0.7421875 , 0.74283854, 0.73763021, 0.73811075, 0.74983713])

In [42]:
round(np.mean(score)*100,2)

74.21

# 예측

In [43]:
# 테스트 데이터를 넣고 예측합니다.
y_pred = forest.predict(test_feature_vector)
y_pred[:3]

array(['X', 'O', 'X'], dtype=object)

In [44]:
y_pred.shape

(854,)

In [45]:
# 예측 결과를 저장하기 위해 데이터프레임에 담아 줍니다.
output = pd.DataFrame(data={'category_pred':y_pred})
output.head(10)

category_pred
0             X
1             O
2             X
3             X
4             X
5             X
6             X
7             X
8             X
9             O

In [46]:
df_test['category_pred'] = y_pred

# 결과검증

In [47]:
df_test['pred_diff'] = 0
df_test['pred_diff'] = (df_test['대상 선정'] == df_test['category_pred'] ) == 1
df_test['pred_diff'] = df_test['pred_diff'].astype(int)
df_test.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd ceq_psit_nm  \
9211               W  2018     20181210         ZZ5         Q57   소음/이음_작동시   
9212               W  2016     20160404         ZZ4         Q57   소음/이음_작동시   
9213               W  2016     20160505         ZZ5         Q57   소음/이음_작동시   
9214               W  2015     20150417         ZZ3         Q57   소음/이음_작동시   
9216               W  2015     20151109         ZZ5         Q57   소음/이음_작동시   

     trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
9211         KV11          YP     A9          TT        98360   
9212         KV11          YP     A9          TT        82210   
9213         KV11          YP     A9          TT        83810   
9214         KV11          YP     A9          TT        92130   
9216         KV11          YP     A9          TT        83820   

                                           clm_prtl_sbc 대상 선정 구분 책임구 원인  검토결과  \
9211        와이퍼작동시소음발생   와이퍼 턴 구간에서턱하는소음발생  와이퍼날 변형으...     X  N   N  N  X---   
9212      운전석 조수석 윈도우 레규레다 하강 간헐적 작동시 이음발생 및   윈도우 스...     X  N   N  N  X---   
9213     슬리이딩 도어 작동시 이음 발생 점검  계속해서 작동해 보니 PSD모터 롤러에...     X  N   N  N  X---   
9214    소음관련 긴급출동으로 출동후 점검시 좌측 헤드램프 레벨링 이큅먼트 액츄에이터 작...     X  N   N  N  X---   
9216      도어 열고 닫을 때  소음발생으로 입고함  점검결과 슬라이딩도어모터에서 이음...     X  N   N  N  X---   

     target_predict category_pred  pred_diff  
9211              X             X          1  
9212              X             O          0  
9213              X             X          1  
9214              X             X          1  
9216              X             X          1

In [48]:
diff_sum = df_test['pred_diff'].sum()
diff_sum

557

In [49]:
y_pred_len = len(y_pred)
y_pred_len

854

# 정확도

In [51]:
Acy = round(diff_sum/y_pred_len*100,1)
Acy

65.2